# Goal: train LSTM model to predict multiple features
- OSLO_temp_mean
- OSLO_cloud_cover
- OSLO_humidity
- OSLO_pressure

In [ ]:
import os
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import numpy as np
import pandas as pd

from datetime import date, datetime
import plotly.graph_objects as go

In [ ]:
os.chdir("..")
os.getcwd()

# Dataframe

In [ ]:
origin_df = pd.read_csv("./resources/weather_prediction_dataset.csv")
base_columns = ["DATE", "MONTH"]
oslo_columns = [x for x in origin_df.columns if x.startswith("OSLO")]
columns = base_columns + oslo_columns
origin_df = origin_df[columns][:-1]

origin_df["YEAR"] = origin_df["DATE"].apply(lambda x: int(str(x)[:4]))
origin_df["DAY"] = origin_df["DATE"].apply(lambda x: int(str(x)[-2:]))
origin_df["DATE"] = [
    date(year=origin_df['YEAR'].iloc[i], month=origin_df['MONTH'].iloc[i], day=origin_df['DAY'].iloc[i])
    for i in range(len(origin_df))
]

origin_df.head()

In [ ]:
stationary_df = pd.read_csv("resources/weather_prediction_stationary_dataset.csv")
stationary_df["DATE"] = pd.to_datetime(stationary_df["DATE"])

In [ ]:
train_features_df = (
    stationary_df
    [stationary_df["DATE"] < datetime(year=2009, month=1, day=1)]
    [["OSLO_temp_mean", "OSLO_cloud_cover", "OSLO_humidity", "OSLO_pressure"]]
)
val_features_df = (
    stationary_df
    [stationary_df["DATE"] >= datetime(year=2009, month=1, day=1)]
    [["OSLO_temp_mean", "OSLO_cloud_cover", "OSLO_humidity", "OSLO_pressure"]]
)

# 3d LSTM data

In [ ]:
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data.iloc[i+seq_length]  # Predict next step
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

In [ ]:
seq_length = 7  # Number of past steps to use for prediction
X_train, y_train = create_sequences(train_features_df, seq_length)
X_val, y_val = create_sequences(val_features_df, seq_length)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

# Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.fc(lstm_out[:, -1, :])  # Use last time step's output

In [ ]:
def train_epoch(model, dataloader, optimizer, criterion, epoch):
    model.train()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    avg_loss = total_loss / len(dataloader)
    
    return model, avg_loss


def val_epoch(model, dataloader, optimizer, criterion, epoch):
    model.eval()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        total_loss += loss.item()
        
    avg_loss = total_loss / len(dataloader)
    
    return avg_loss


def early_stoppage(loss_history: list[tuple[float, float]], min_incr: float, last_epochs: int = 3) -> bool:
    stop = False
    val_loss_data = [x[1] for x in loss_history]
    if len(loss_history) > last_epochs:
        last_results = np.mean(val_loss_data[-last_epochs-1:])
        diff = last_results - val_loss_data[-1]
        if diff < min_incr:
            stop = True
            print("Early stoppage!")
    return stop
            

def train_and_validate(model, num_epochs, train_dataloader, val_dataloader, optimizer, criterion, min_incr):
    loss_history = []
    for epoch in range(num_epochs):
        model, train_loss = train_epoch(model, train_dataloader, optimizer, criterion, epoch)
        val_loss = val_epoch(model, val_dataloader, optimizer, criterion, epoch)
        loss_history.append((train_loss, val_loss))
        print(f"Epoch {epoch+1}/{num_epochs}: train mse = {round(train_loss, 3)}, val mse = {round(val_loss, 3)}")
        if early_stoppage(loss_history, min_incr):
            break
    return model, loss_history

In [ ]:
input_dim = X_train.shape[2]  # Number of features (4 in this case)
hidden_dim = 20
num_layers = 4
output_dim = X_train.shape[2]  # Predicting next step for all features
batch_size = 7 # one week
min_incr = 0.001
lr = 0.0001
num_epochs = 1000

In [ ]:
model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

train_dataloader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=False)
val_dataloader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

In [ ]:
model, loss_history = train_and_validate(model, num_epochs, train_dataloader, val_dataloader, optimizer, criterion, min_incr)

In [ ]:
fig = go.Figure()

x = [i+1 for i in range(num_epochs)]
fig.add_trace(
    go.Scatter(
        name="Train MSE",
        x=x,
        y=[x[0] for x in loss_history],
        # mode="lines"
    )
)
fig.add_trace(
    go.Scatter(
        name="Val MSE",
        x=x,
        y=[x[1] for x in loss_history],
        # mode="lines"
    )
)

fig.update_layout(
    title="Train history",
    xaxis_title="Epoch",
    yaxis_title="Loss",
    width=800,
    height=700,
)

fig.show()

# Save

In [ ]:
torch.save(model.state_dict(), "resources/lstm")